In [1]:
from brian_graph import *
from solver import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
solve_from_file("inputs/265_50.in", "outputs", ["ILP"])

Processing inputs/84_50.in
Academic license - for non-commercial use only
Optimize a model with 110 rows, 2835 columns and 5582 nonzeros
Model has 33660 quadratic objective terms
Variable types: 0 continuous, 2835 integer (2835 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  QObjective range [2e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 501.8000000
Presolve removed 33 rows and 1395 columns
Presolve time: 0.02s
Presolved: 695 rows, 2058 columns, 33816 nonzeros
Variable types: 0 continuous, 2058 integer (1441 binary)

Root relaxation: objective 9.400000e+00, 155 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    9.40000    0   36  501.80000    9.40000  98.1%     -    0s
H    0     0                     301.9000000    9.4000

 149606 121125   90.51703  228   46  119.30000   23.62053  80.2%  17.2  305s
 152782 123926   42.66412  103   49  119.30000   23.65519  80.2%  17.1  310s
 155082 125987   44.80000  116   25  119.30000   23.66802  80.2%  17.2  315s
 158632 129093   50.58659  102   40  119.30000   23.70884  80.1%  17.2  320s
 162224 132424   76.55918  182   46  119.30000   23.73820  80.1%  17.2  326s
 166012 135860   29.48017   89   58  119.30000   23.77024  80.1%  17.2  330s
 169641 139176   47.67956   87   48  119.30000   23.81323  80.0%  17.2  335s
 173093 142208   50.46500   93   49  119.30000   23.84900  80.0%  17.2  340s
 176657 145303   45.53454   61   41  119.30000   23.88433  80.0%  17.2  345s
 180229 148547   79.15333  188   45  119.30000   23.92794  79.9%  17.2  350s
 184249 152235   46.35779  100   62  119.30000   23.98081  79.9%  17.2  356s
 187644 155341   87.56628  157   44  119.30000   24.02278  79.9%  17.1  360s
 190436 157843   67.12184  145   46  119.30000   24.07153  79.8%  17.1  365s